SCADA EDA

needed libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
import dask.dataframe as dd
import pyarrow as pa
print(pa.__version__)

19.0.0


Load data and define file paths

In [3]:


# Define the base directory for raw data
base_path = r"C:\Users\Linds\Repos\East_River\Data\Raw"

In [4]:
# Define file paths
file_paths = {
    "2021 Control Threshold": os.path.join(base_path, "2021 Control Threshold.csv"),
    "2021 Load Control": os.path.join(base_path, "2021 Load Control.csv"),
    "2021 Online Load": os.path.join(base_path, "2021 Online Load.csv"),
    "2022 Control Threshold": os.path.join(base_path, "2022 Control Threshold.csv"),
    "2022 Load Control": os.path.join(base_path, "2022 Load Control.csv"),
    "2022 Online Load": os.path.join(base_path, "2022 Online Load.csv"),
    "2023 Control Threshold": os.path.join(base_path, "2023 Control Threshold.csv"),
    "2023 Load Control": os.path.join(base_path, "2023 Load Control.csv"),
    "2023 Online Load": os.path.join(base_path, "2023 Online Load.csv"),
    "2024 Control Threshold": os.path.join(base_path, "2024 Control Threshold.csv"),
    "2024 Load Control": os.path.join(base_path, "2024 Load Control.csv"),
    "2024 Online Load": os.path.join(base_path, "2024 Online Load.csv"),
}


In [5]:
# Verify that all files exist before loading
missing_files = [name for name, path in file_paths.items() if not os.path.exists(path)]
if missing_files:
    print(f"⚠️ Warning: The following files are missing: {missing_files}")
else:
    print("✅ All files found. Proceeding with loading...")

✅ All files found. Proceeding with loading...


In [6]:
# Function to process each year's data
def process_year(year):
    files = {
        "Control Threshold": f"{year} Control Threshold.csv",
        "Load Control": f"{year} Load Control.csv",
        "Online Load": f"{year} Online Load.csv",
    }
    
    # Load datasets
    dfs = {}
    for key, file in files.items():
        path = os.path.join(base_path, file)
        if os.path.exists(path):
            df = pd.read_csv(path, parse_dates=["Timestamp"])
            df.rename(columns={df.columns[1]: key}, inplace=True)
            dfs[key] = df
        else:
            print(f"⚠️ Missing: {path}")

    # Merge three datasets for the year
    merged_year = dfs["Control Threshold"]
    merged_year = merged_year.merge(dfs["Load Control"], on="Timestamp", how="outer")
    merged_year = merged_year.merge(dfs["Online Load"], on="Timestamp", how="outer")

    # Save interim file
    output_file = os.path.join(base_path, f"Merged_{year}.csv")
    merged_year.to_csv(output_file, index=False)
    print(f"✅ {year} merged and saved as {output_file}")

# Process all years
for y in [2021, 2022, 2023, 2024]:
    process_year(y)

✅ 2021 merged and saved as C:\Users\Linds\Repos\East_River\Data\Raw\Merged_2021.csv
✅ 2022 merged and saved as C:\Users\Linds\Repos\East_River\Data\Raw\Merged_2022.csv
✅ 2023 merged and saved as C:\Users\Linds\Repos\East_River\Data\Raw\Merged_2023.csv
✅ 2024 merged and saved as C:\Users\Linds\Repos\East_River\Data\Raw\Merged_2024.csv


In [17]:
import pandas as pd
import os

# Define base path
base_path = r"C:\Users\Linds\Repos\East_River\Data\Raw"

# List of yearly merged files
year_files = [
    os.path.join(base_path, "Merged_2021.csv"),
    os.path.join(base_path, "Merged_2022.csv"),
    os.path.join(base_path, "Merged_2023.csv"),
    os.path.join(base_path, "Merged_2024.csv"),
]

# Output file path
final_output = os.path.join(base_path, "Final_Merged_Data.csv")
temp_output = final_output + ".tmp"
if os.path.exists(temp_output):
    os.remove(temp_output)
if os.path.exists(final_output):
    try:
        os.remove(final_output)
    except PermissionError:
        print(f"⚠️ Could not remove {final_output}. It may be open in another program. Writing to temporary file instead.")
        final_output = temp_output

# Open first file and write to output in chunks
chunk_size = 500_000  # Adjust based on available RAM

print("📥 Starting disk-based merging process...")

with open(final_output, "w", newline="") as f_out:
    # Process the first file
    first_file = year_files[0]
    print(f"🔹 Writing {first_file} as the base dataset...")
    
    for chunk in pd.read_csv(first_file, chunksize=chunk_size):
        chunk.to_csv(f_out, index=False, mode="w", header=True)

# If we wrote to a temporary file due to permission issues, attempt to rename it back.
if final_output.endswith(".tmp"):
    try:
        os.replace(temp_output, final_output[:-4])
        final_output = final_output[:-4]
    except PermissionError:
        print(f"⚠️ Temporary file {temp_output} could not be renamed. Please ensure it is not open in another program.")

print(f"✅ Final dataset saved at: {final_output}")

for file in year_files[1:]:
    print(f"🔹 Merging {file} in chunks...")
    with open(final_output, "a", newline="") as f_out:
        for chunk in pd.read_csv(file, chunksize=chunk_size):
            chunk.to_csv(f_out, index=False, mode="a", header=False)

print(f"✅ Final dataset saved at: {final_output}")


⚠️ Could not remove C:\Users\Linds\Repos\East_River\Data\Raw\Final_Merged_Data.csv. It may be open in another program. Writing to temporary file instead.
📥 Starting disk-based merging process...
🔹 Writing C:\Users\Linds\Repos\East_River\Data\Raw\Merged_2021.csv as the base dataset...
⚠️ Temporary file C:\Users\Linds\Repos\East_River\Data\Raw\Final_Merged_Data.csv.tmp could not be renamed. Please ensure it is not open in another program.
✅ Final dataset saved at: C:\Users\Linds\Repos\East_River\Data\Raw\Final_Merged_Data.csv.tmp
🔹 Merging C:\Users\Linds\Repos\East_River\Data\Raw\Merged_2022.csv in chunks...
🔹 Merging C:\Users\Linds\Repos\East_River\Data\Raw\Merged_2023.csv in chunks...
🔹 Merging C:\Users\Linds\Repos\East_River\Data\Raw\Merged_2024.csv in chunks...
✅ Final dataset saved at: C:\Users\Linds\Repos\East_River\Data\Raw\Final_Merged_Data.csv.tmp


In [21]:


file_path = r"C:\Users\Linds\Repos\East_River\Data\Raw\Final_Merged_Data.csv.tmp"

df = pd.read_csv(file_path) 
print(df.head())  # Check structure

             Timestamp  Control Threshold  Load Control  Online Load
0  2021-01-01 00:00:00              626.2           0.0       514.37
1  2021-01-01 00:30:00              572.0           0.0       505.76
2  2021-01-01 01:00:00              572.0           0.0       504.80
3  2021-01-01 01:30:00              572.0           0.0       499.74
4  2021-01-01 02:00:00              572.0           0.0       496.19


In [22]:
# Check the total number of rows
print(f"Total rows: {len(df)}")

# Check for missing values in each column
missing_values = df.isnull().sum()

# Display columns with missing values
print("\nColumns with missing values:")
print(missing_values[missing_values > 0])

Total rows: 455236

Columns with missing values:
Timestamp            385112
Control Threshold    368368
Load Control         368368
Online Load          368424
dtype: int64


In [24]:
# Check if all columns have the same number of non-null values
row_counts = df.notnull().sum()

print("\nRow count per column:")
print(row_counts)

# Check if all values match the expected row count
if len(set(row_counts)) == 1:
    print("✅ All columns have the same number of rows!")
else:
    print("⚠️ WARNING: Some columns have missing values!")



Row count per column:
Timestamp            70124
Control Threshold    86868
Load Control         86868
Online Load          86812
dtype: int64
⚠️ WARNING: Some columns have missing values!


In [25]:
# Count duplicate timestamps
duplicate_timestamps = df["Timestamp"].duplicated().sum()

if duplicate_timestamps > 0:
    print(f"⚠️ Warning: {duplicate_timestamps} duplicate timestamps found!")
else:
    print("✅ No duplicate timestamps found. Data alignment looks good.")

⚠️ Warning: 385114 duplicate timestamps found!


In [26]:
# Save the temporary file as a proper CSV
final_output = r"C:\Users\Linds\Repos\East_River\Data\Raw\Final_Merged_Data.csv"

# Rename the temp file if it exists
temp_file = final_output + ".tmp"

import os
import shutil

if os.path.exists(temp_file):
    try:
        shutil.move(temp_file, final_output)
        print(f"✅ Successfully renamed temp file to: {final_output}")
    except PermissionError:
        print(f"⚠️ Could not rename {temp_file}. Ensure it's not open elsewhere.")
else:
    print(f"⚠️ Temporary file {temp_file} not found. Please check the directory.")

✅ Successfully renamed temp file to: C:\Users\Linds\Repos\East_River\Data\Raw\Final_Merged_Data.csv
